# Complete LLM Cheatsheet with RAG and Agents (using Langgraph)

- LLM APIs
- Huggingface Embedding Models
- Data Loaders
- Chunking
- Vector Embedding using FAISS, Chroma, and Pinecone
- Langchain Inbuilt Tools
- Creating custom tools
- Agentic Orchestration
- ReAct Agents
- Agentic RAG
- MultiAgents (Network and Supervisor)
- Human in Loop and Misc. Manipulations with Flow

## Loading Env

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

## LLM APIs

### OpenAI Env, Model, and Embedding

In [ ]:
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

llm.invoke("hello how are you my firend?")

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

len(embeddings.embed_query("hello how are you my firend?"))

### Groq KEY and Model

In [2]:
import os
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="deepseek-r1-distill-llama-70b",
    temperature=0
)

response=llm.invoke("what is length of wall of china?")

### Google Gemini Env, Model, and Embedding

In [ ]:
import os
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

output = model.invoke("hi")
print(output.content)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

embeddings.embed_query("Hello AI")

## Hugging Face Embedding Models

In [ ]:
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")

len(embeddings.embed_query("hi"))

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

text="this is atest documents"
query_result=embeddings.embed_query(text)
query_result

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

## Data Loaders

### WebBaseLoader

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

# For 1 URL
url = ''
web_loader=WebBaseLoader(url)
data=web_loader.load()

# For Multi URL
urls = ['', '']
docs=[WebBaseLoader(url).load() for url in urls]
docs_list=[item for sublist in docs for item in sublist]

### TextLoader and DirectoryLoader

In [ ]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader

loader=DirectoryLoader("../data",glob="./*.txt",loader_cls=TextLoader)
docs=loader.load()

### PDF Loader

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader('syllabus.pdf')
docs=loader.load()

### ArXiv Loader

In [ ]:
from langchain_community.document_loaders import ArxivLoader

docs = ArxivLoader(query="1706.03762", load_max_docs=2).load()

### Wikipedia Loader

In [ ]:
from langchain_community.document_loaders import WikipediaLoader

docs = WikipediaLoader(query="Generative AI", load_max_docs=4).load()

## Chunking

### RecursiveCharaterTextSplitter

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Normal Embedding Models
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)

# OpenAI Embedding Models
text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder
(
    chunk_size=100,
    chunk_overlap=25
)

# Common Code
doc_splits=text_splitter.split_documents(docs_list)


# If only page content needed
doc_string=[doc.page_content for doc in doc_splits]

# If need to preserve metadata
texts = [doc.page_content for doc in doc_splits]
metadatas = [doc.metadata for doc in doc_splits]

### CharacterTextSplitter

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter=CharacterTextSplitter(separator="\n\n",chunk_size=100,chunk_overlap=20)
text_splitter.split_documents(docs)

### HTMLHeaderTextSplitter

In [ ]:
from langchain_text_splitters import HTMLHeaderTextSplitter

html_string = """
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>Foo</h1>
        <p>Some intro text about Foo.</p>
        <div>
            <h2>Bar main section</h2>
            <p>Some intro text about Bar.</p>
            <h3>Bar subsection 1</h3>
            <p>Some text about the first subtopic of Bar.</p>
            <h3>Bar subsection 2</h3>
            <p>Some text about the second subtopic of Bar.</p>
        </div>
        <div>
            <h2>Baz</h2>
            <p>Some text about Baz</p>
        </div>
        <br>
        <p>Some concluding text about Foo</p>
    </div>
</body>
</html>
"""

headers_to_split_on=[
    ("h1","Header 1"),
    ("h2","Header 2"),
    ("h3","Header 3")
]

html_splitter=HTMLHeaderTextSplitter(headers_to_split_on)
html_header_splits=html_splitter.split_text(html_string)
html_header_splits

### RecursiveJsonSplitter

In [ ]:
from langchain_text_splitters import RecursiveJsonSplitter

json_splitter=RecursiveJsonSplitter(max_chunk_size=300)
json_chunks=json_splitter.split_json(json_data)

## Vector Embedding

### FAISS

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# Using Inner Product in FAISS Index

index=faiss.IndexFlatIP(3072) # Number of dimensions in the embedding model

db = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# Using Euclidiean Distance in FAISS Index

index=faiss.IndexFlatL2(384) # Number of dimensions in the embedding model

db = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# If we just need the docstrings
db.add_texts(doc_string)

# If we need to add metadata info as well
db.add_texts(texts, metadatas=metadatas)

# Note: Add texts only works with array of docs

### Saving and Loading Indexes

In [ ]:
# Saving Index

db.save_local("saved_index")

In [ ]:
# Loading Index

new_vector_store=FAISS.load_local(
  "saved_index",
  embeddings,
  allow_dangerous_deserialization=True
)

### Chroma

In [ ]:
from langchain_community.vectorstores import Chroma

vectorstore=Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chrome", # Any Name
    embedding=embeddings
    
)

### Pinecone

In [ ]:
import os
pinecone_api_key=os.getenv("PINECONE_API_KEY")

In [ ]:
from pinecone import Pinecone
from pinecone import ServerlessSpec  #Serverless: Server will be Managed by the cloud provider

pc=Pinecone(api_key=pinecone_api_key)

# Index Creation and Loading

index_name="agentic-ai"

#creating a index
if not pc.has_index(index_name):
    pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws",region="us-east-1")    
)

#loading the index
index=pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

# Vector Store and Similarity Search
vector_store=PineconeVectorStore(index=index,embedding=embeddings)

results = vector_store.similarity_search("what is a langchain?")
results

In [ ]:
# Vector Store Retriever

retriever=vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7} #hyperparameter
)
retriever.invoke("langchain")

## Langchain Inbuilt Tools

### Wikipedia

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper=WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max= 500)
wiki_tool = WikipediaQueryRun(api_wrapper= api_wrapper)

# To get tool name
wiki_tool.name
# To get tool description
wiki_tool.description
# To get tool args
wiki_tool.args

# Running
wiki_tool.run({"query": "elon musk"})

### Youtube Search

In [ ]:
from langchain_community.tools import YouTubeSearchTool

tool = YouTubeSearchTool()

# To get tool name
tool.name
# To get tool description
tool.description
# To get tool args
tool.args

# Running
tool.run("Emergency Awesome")

### Tavily (Search Engine)

In [ ]:
import os
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool=TavilySearchResults(tavily_api_key=TAVILY_API_KEY)

# Running - 1
tool.invoke({"query":"what happend between Trump and Musk today?"})

# Running - 2
question = "what happend between Trump and Musk today?"
complete_query = "Anwer the follow question by searching the internet and getting best response. Following is the user question: " + question

tool.invoke(complete_query)

In [ ]:
from langchain_tavily import TavilySearch

tavily_tool=TavilySearch(tavily_api_key=TAVILY_API_KEY)

question = "what happend between Trump and Musk today?"
complete_query = "Anwer the follow question by searching the internet and getting best response. Following is the user question: " + question

tavily_tool.invoke(complete_query)

### DuckDuckGo

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("what is the latest update on iphone17 release?")

### Python REPL Utililty

In [ ]:
# Run any given python code

from langchain_experimental.utilities import PythonREPL

repl = PythonREPL()

code = """
x = 5
y = x * 2
print(y)
"""

repl.run(code)

## Custom Tools

### Addition

In [ ]:
from langchain.tools import tool

@tool
def add(a: int, b: int) -> int:
    """
    Add two integers.

    Args:
    a(int): The first integer
    b(int): The second integer

    Returns:
        int: The Sum of a and b
    """

    return a + b

### Subtract

In [ ]:
from langchain.tools import tool

@tool
def subtract(a: int, b: int) -> int:
    """
    Subtract two integers.

    Args:
    a(int): The first integer
    b(int): The second integer

    Returns:
        int: The difference of a and b
    """

    return a - b

### Absolute Difference

In [ ]:
from langchain.tools import tool

@tool
def abs_diff(a: int, b: int) -> int:
    """
    Subtract two integers.

    Args:
    a(int): The first integer
    b(int): The second integer

    Returns:
        int: The absolute difference of a and b
    """

    return abs(a - b)

### Multiplication

In [ ]:
from langchain.tools import tool

@tool
def multiple(a: int, b: int) -> int:
    """
    Multiple two integers.

    Args:
    a(int): The first integer
    b(int): The second integer

    Returns:
        int: The product of a and b
    """

    return a * b

### Divide

In [ ]:
from langchain.tools import tool

@tool
def divide(a: int, b: int) -> int:
    """
    Divide two integers.

    Args:
    a(int): The first integer
    b(int): The second integer

    Returns:
        int: The result of division
    """

    if b == 0:
        raise ValueError("Denominator cannot be zero.")
    return a / b

### Length of Word

In [ ]:
@tool
def get_word_length(word:str)->int:
    """
    Calculate the length of the word.

    Args:
    word(str): The word in string

    Returns:
        int: The length of the word
    """
    return len(word)

## Agentic Orchestration

### Pydantic Class for some kind of validation -> used as an Output Parser

In [ ]:
from pydantic import BaseModel , Field
from langchain.output_parsers import PydanticOutputParser

class TopicSelectionParser(BaseModel):
    Topic:str=Field(description="selected topic")
    Reasoning:str=Field(description='Reasoning behind topic selection')

parser=PydanticOutputParser(pydantic_object=TopicSelectionParser)
parser.get_format_instructions()

### Custom Agent State Initiation

In [ ]:
import operator
from langchain_core.messages import BaseMessage
from typing import TypedDict, Annotated, Sequence

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [ ]:
from langgraph.graph import StateGraph

workflow = StateGraph(AgentState)

### Prebuilt Agent State

In [ ]:
from langgraph.graph import StateGraph, MessagesState

workflow = StateGraph(MessagesState) 
# This is the same as our custom defined Agent State Function (right now), if we need something custom, we can use our methods, else MessageState is better

### Workflow 1: Agentic Orchestration using parser(pydantic class), custom AgentState, and Custom Router Function

![alt text](01a93893-3dce-4f90-80d0-b335c9bd36a2.png)

In [ ]:
from typing import List
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END

In [ ]:
# Functions

# LLM Supervisor Function using Pydantic Parser, Custom Agent State, Chaining
def llm_supervisor_function(state: AgentState):
    question = state["messages"][-1]

    print("Question", question)

    template="""
    Your task is to classify the given user query into one of the following categories: [USA, Not Related]. 
    Only respond with the category name and nothing else.

    User query: {question}
    {format_instructions}
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["question"],
        partial_variables={"format_instructions": parser.get_format_instructions}
    )

    chain = prompt | model | parser

    response = chain.invoke({"question": question})

    print("Parsed response", response)

    return {"messages": [response.Topic]}

# Custom Router Function
def router_function(state: AgentState):
    print("-> Router ->")

    last_message = state["messages"][-1]
    print("last_message: ", last_message)

    if "usa" in last_message.lower():
        return "RAG Call"
    else:
        return "LLM Call"
    
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# RAG Function
def function2(state: AgentState):
    print("-> RAG Call ->")
    question = state["messages"][0]
    
    prompt=PromptTemplate(
        template = """
        You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
        If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n
        Question: {question} \n
        Context: {context} \n
        Answer:
        """,
        
        input_variables=['context', 'question']
    )
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    result = rag_chain.invoke(question)
    return  {"messages": [result]}

# LLM Function
def function3(state: AgentState):
    print("-> LLM Call ->")
    question = state["messages"][0]
    
    # Normal LLM call
    complete_query = "Anwer the follow question with you knowledge of the real world. Following is the user question: " + question
    response = model.invoke(complete_query)
    return {"messages": [response.content]}

In [ ]:
workflow.add_node("Supervisor", llm_supervisor_function)
workflow.add_node("RAG", function2)
workflow.add_node("LLM", function3)

# One Way
workflow.set_entry_point("Supervisor")

# Other Way
workflow.add_edge(START, "Supervisor")

# When Conditional Edges
workflow.add_conditional_edges(
    "Supervisor",
    router_function,
    {
        "RAG Call" : "RAG",
        "LLM Call" : "LLM"
    }
)

workflow.add_edge("RAG", END)
workflow.add_edge("LLM", END)

app = workflow.compile()
app

### Workflow 2: Agentic Orchestration Using Message State, Tool Node, Custom Tools, Multi Tool Calls

![alt text](95f33936-ba48-4300-9742-488a7f1cd6f3.png)

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph,MessagesState,START,END
from langgraph.prebuilt import ToolNode

In [ ]:
# Custom Tool

@tool
def search(query:str):
    """this is my custom tool for searching a weather"""
    if "delhi" in query.lower():
        return "the temp is 45 degree celsius"
    return "the temp is 25 degree celsius"

# Binding Tool
tools = [search]
llm_with_tool = llm.bind_tools(tools)

response = llm_with_tool.invoke("what is weather in delhi?")
response.tool_calls

In [ ]:
# Using Message State

def call_model(state: MessagesState):
    question = state['messages']
    response = llm_with_tool.invoke(question)
    return {"messages": [response]}

# Custom Router Function, which will be replaced by langgraph.prebuilt tools_condition 
def router_function(state: MessagesState):
    message = state["messages"]
    last_message = message[-1]

    if last_message.tool_calls:
        return "tools"
    return END

In [ ]:
# Creating tool Node from Tool Node class

tool_node = ToolNode(tools)

In [ ]:
workflow2 = StateGraph(MessagesState)
workflow2.add_node("llmwithtool",call_model)
workflow2.add_node("mytools", tool_node) # The tool Node is used here
workflow2.add_edge(START, "llmwithtool")
workflow2.add_conditional_edges("llmwithtool",
                                router_function,
                                {"tools": "mytools",
                                END: END})
workflow2.add_edge("mytools", "llmwithtool") # This edge makes multi tool call
app2 = workflow2.compile()
app2

### Using tools_condition instead of our custom Router function

In [ ]:
# Instead of using router_function in the add_condition_edges method, use tools_condition. This is a inbuilt function in langgraph which returns "tools" (same as in our router function)

from langgraph.prebuilt import tools_condition

workflow.add_conditional_edges("llmwithtool",
                            tools_condition)

### Use of Memory Saver, Stream Messages, and Pretty Print

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Only need to add checkpointer to this memory 
app2 = workflow2.compile(checkpointer=memory)
app2

In [ ]:
config = {"configurable": {"thread_id": "1"}}

# Creating this events 
events = app2.stream(
    {"messages":["what is a weather in delhi can you tell me some good hotel for staying in north delhi"]}, 
    config=config, 
    stream_mode="values"
    )

for event in events:
    event["messages"][-1].pretty_print()

### Viewing Compiled Workflow stored in app variable

In [ ]:
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))

### Agentic RAG

![alt text](004a619e-f184-41ff-b629-ecdad04b9eb0.png)

In [ ]:
# Data Retrieval Step until Vector Embedding creation and adding the doc list to it

retriever=vectorstore.as_retriever()

In [ ]:
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import ToolNode

# Adding retriever as a tool
retriever_tool=create_retriever_tool(
    retriever,
    "retriever_blog_post", # Any Name
    "vector database data description, comprehensive for the model to make sense of", # Description for vector db data
    )

# Adding to tools and Tool Node Creation
tools=[retriever_tool]
llm_with_tool=llm.bind_tools(tools)

retriever_node = ToolNode(tools)

In [ ]:
from typing import Annotated,Sequence, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# Pydantic Class 
class grade(BaseModel):
    binary_score:str=Field(description="Relvance score 'yes' or 'no'")

In [ ]:
# Functions
from langchain import hub
from typing import Literal
#we use it for type of hinting

def LLM_Decision_Maker(state:AgentState):
    print("----CALL LLM_DECISION_MAKE----")
    message=state["messages"]
    last_message=message[-1]
    question=last_message.content
    response=llm_with_tool.invoke(question)
    return {"messages":[response]}


def grade_documents(state:AgentState)->Literal["Output Generator", "Query Rewriter"]:
    print("----CALLING GRADE FOR CHECKING RELEVANCY----")
    llm_with_structure_op=llm.with_structured_output(grade) # To provide response in a certain assigned schema (grade here)
    
    prompt=PromptTemplate(
        template="""You are a grader deciding if a document is relevant to a user’s question.
                    Here is the document: {context}
                    Here is the user’s question: {question}
                    If the document talks about or contains information related to the user’s question, mark it as relevant. 
                    Give a 'yes' or 'no' answer to show if the document is relevant to the question.""",
                    input_variables=["context", "question"]
                    )
     
    chain=prompt|llm_with_structure_op
     
     
    message=state['messages']
    
    last_message = message[-1]
    
    question = message[0].content
    
    docs = last_message.content
    
    scored_result=chain.invoke({"question": question, "context": docs})
    
    score=scored_result.binary_score
     
    if score=="yes":
        print("----DECISION: DOCS ARE RELEVANT----")
        return "generator"
    else:
        print("----DECISION: DOCS ARE NOT RELEVANT----")
        return "rewriter"
    

def generate(state:AgentState):
    print("----RAG OUTPUT GENERATE----")
    
    message=state["messages"]
    question=message[0].content
    
    last_message = message[-1]
    docs = last_message.content

    # To assign a said prompt for the RAG part of code using the preexisting library
    prompt=hub.pull("rlm/rag-prompt")
    
    rag_chain=prompt | llm
    
    response=rag_chain.invoke({"context": docs, "question": question})
    
    print(f"this is my response:{response}")
    
    return {"messages": [response]}

def rewrite(state:AgentState):
    print("----TRANSFORM QUERY----")
    message=state["messages"]
    
    question=message[0].content
    
    input= [HumanMessage(content=f"""Look at the input and try to reason about the underlying semantic intent or meaning. 
                    Here is the initial question: {question} 
                    Formulate an improved question: """)
       ]

    response=llm.invoke(input)
    
    return {"messages": [response]}

In [ ]:
# Agentic Orchestration

from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import tools_condition

workflow=StateGraph(AgentState)

workflow.add_node("LLM Decision Maker",LLM_Decision_Maker)
workflow.add_node("Vector Retriever",retriever_node)
workflow.add_node("Output Generator",generate)
workflow.add_node("Query Rewriter",rewrite)

workflow.add_edge(START,"LLM Decision Maker")
workflow.add_conditional_edges("LLM Decision Maker",
                               tools_condition,
                               {"tools":"Vector Retriever",
                                END:END
                                })
workflow.add_conditional_edges("Vector Retriever",
                               grade_documents,
                               {"generator":"Output Generator",
                                "rewriter":"Query Rewriter"
                                })
workflow.add_edge("Output Generator",END)
workflow.add_edge("Query Rewriter","LLM Decision Maker")

app=workflow.compile()
app

### Invoking created Agentic Apps

In [ ]:
app.invoke({"messages":["what is LLM Powered Autonomous Agents explain the planning and reflection and prompt engineering explain me in terms of agents and langchain?"]})

## MultiAgents

#### Concept of Command in MultiAgents

In [ ]:
from langgraph.types import Command

def add_number(state):
    result = state["num1"] + state["num2"]
    print(f"Addition is {result}")

    return Command(goto="multiply", update = {"sum": result}) # Updates the current state, and tells which Agent to go to Next

state = {"num1": 10, "num2": 20}

add_number(state)

Addition is 30


Command(update={'sum': 30}, goto='multiply')

#### Concept of create_react_agent to use inbuilt library to create an agent 

In [ ]:
from langgraph.prebuilt import create_react_agent

# Creates an Agent, with defined LLM, tools, and the PROMPT we provide

research_agent = create_react_agent(
        llm,
        tools = [tools_arr],
        prompt = "PROMPT FOR THE AGENT"
    )

### Network/Collaboration MultiAgent

![alt text](b9c5dea6-b44a-4c74-a92f-9d2642bbd17b.png)

In [ ]:
from typing_extensions import Literal
from typing import Annotated
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langgraph.graph import MessagesState,StateGraph, START,END
from langchain_experimental.utilities import PythonREPL
from langchain_core.messages import BaseMessage, HumanMessage


#### Tools and Function Creation

In [ ]:
# Python REPL Tool Creation to run the code it recieves

@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
    ):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    
    try:
        result = repl.run(code)
        
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    
    result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )


# A system prompt to tell LLM, adding agent vise context
def make_system_prompt(instruction:str)->str:
    return  (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{instruction}"
    )


# Directing the LLM, what to do based on the last_message
def get_next_node(last_message: BaseMessage, goto: str):
    if "FINAL ANSWER" in last_message:
        return END
    
    return goto

In [ ]:
# Agent 1 -> Researcher Node

def research_node(state: MessagesState)-> Command[Literal["chart_generator", END]]:
    research_agent = create_react_agent(
        llm,
        tools = [search_tool],
        prompt = make_system_prompt(
            "You can only do research. You are working with a chart generator colleague."
        ),
    )

    result = research_agent.invoke(state)

    last_message = result["messages"][-1]

    goto=get_next_node(last_message,"chart_generator")

    result["messages"][-1] = HumanMessage(content=result["messages"][-1].content, name="researcher")

    return Command(update= {"messages": result["messages"]}, goto = goto)


# Agent 2 -> Chart Creation Node

def chart_node(state: MessagesState)-> Command[Literal["researcher", END]]:
    chart_agent = create_react_agent(
        llm,
        tools = [python_repl_tool],
        prompt = make_system_prompt(
            "You can only generate charts. You are working with a researcher colleague."
        ),
    )

    result = chart_agent.invoke(state)

    last_message = result["messages"][-1]

    goto=get_next_node(last_message,"researcher")

    result["messages"][-1] = HumanMessage(content=result["messages"][-1].content, name="chart_generator")

    return Command(update= {"messages": result["messages"]}, goto= goto)

#### Workflow

In [ ]:
workflow = StateGraph(MessagesState)

workflow.add_node("researcher", research_node)
workflow.add_node("chart_generator", chart_node)

workflow.add_edge(START, "researcher")
app = workflow.compile()

app

In [ ]:
app.invoke({"messages": [("user","get the UK's GDP over the past 3 years, then make a line chart of it.Once you make the chart, finish.")],})

### Supervisor MultiAgent

![alt text](a3a4db77-825b-49e9-abdc-ff0bb7255e37.png)

#### Tools and Function Creation

In [ ]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults
from typing import Annotated
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL
from typing import Literal
from typing_extensions import TypedDict
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage, HumanMessage

In [ ]:
# Tool 1 - Researcher
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
search_tool=TavilySearchResults(tavily_api_key=TAVILY_API_KEY)


# Tool 2 - Coder
repl=PythonREPL()

@tool
def python_repl_tool(code: Annotated[str, "The python code to execute to generate your chart."]):
    """Use this to execute python code and do math. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    
    result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
    return result_str

In [ ]:
members=["researcher","coder"]

options = members+["FINISH"]

class Router(TypedDict):
    next: Literal['researcher', 'coder', 'FINISH']

#### IMPORTANT
class State(MessagesState):
    next: str

# State will look like this
# state={"messages": ["hi"], "next": "research_agent"}

system_prompt = f""""
You are a supervisor, tasked with managing a conversation between the following workers: {members}. 
Given the following user request, respond with the worker to act next. 
Each worker will perform a task and respond with their results and status. 
When finished, respond with FINISH.
"""

In [ ]:
# AGENT 1 [SUPERVISOR]

def supervisor_agent(state: State)->Command[Literal['researcher', 'coder', '__end__']]:
    messages = [{"role": "system", "content": system_prompt}, ] + state["messages"]

    llm_with_structured_output = llm.with_structured_output(Router)

    response = llm_with_structured_output.invoke(messages)

    # this is my next worker agent
    goto = response["next"]

    print("***************BELOW IS MY GOTO****************")

    print(goto)

    if goto == "FINISH":
        return '__end__'
    
    return Command(goto= goto, update={"next": goto})

In [ ]:
# AGENT 2 [RESEARCHER]

def researcher_agent(state: State)->Command[Literal['supervisor']]:
    research_agent = create_react_agent(llm, tools=[search_tool], prompt="You are a researcher. DO NOT do any math.")

    result = research_agent.invoke(state)

    return Command(
        update = {
            "messages": [
                HumanMessage(content = result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor"
    )

In [ ]:
# AGENT 3 [CODER]

def coder_agent(state: State)->Command[Literal['supervisor']]:
    code_agent = create_react_agent(llm, tools=[python_repl_tool], prompt="You are a coder. DO NOT do any research.")

    result = code_agent.invoke(state)

    return Command(
        update = {
            "messages": [
                HumanMessage(content = result["messages"][-1].content, name = "coder")
            ]
        },
        goto="supervisor"
    )

#### Workflow

In [ ]:
graph = StateGraph(State)

graph.add_node("supervisor", supervisor_agent)
graph.add_node("researcher", researcher_agent)
graph.add_node("coder", coder_agent)

graph.add_edge(START, "supervisor")

app = graph.compile()

app

In [ ]:
for s in app.stream({"messages": [("user", "What's the square root of 49?")]}, subgraphs=True):
    print(s)
    print("**********BELOW IS MY STATE***************")

## Human In Loop and Misc. Agent Concepts

#### Tools

In [6]:
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

In [7]:
@tool
def multiply(x: int, y: int) -> int:
    """Multiplies two numbers."""
    return x * y

@tool
def search(query: str):
    """search the web for a query and return the results"""
    tavily=TavilySearchResults()
    result=tavily.invoke(query)
    return f"Result for {query} is: \n{result}"


tools = [multiply, search]
llm_with_tools=llm.bind_tools(tools)

#### New Concept: Handling Tools and Understanding Tool Mapping

In [8]:
result=llm_with_tools.invoke("what is current gdp of india?")

In [9]:
result.tool_calls

[{'name': 'search',
  'args': {'query': 'current GDP of India'},
  'id': 'p5t52ta10',
  'type': 'tool_call'}]

In [10]:
result.tool_calls[0]["name"]

'search'

In [11]:
result.tool_calls[0]["args"]

{'query': 'current GDP of India'}

In [12]:
# Tool Mapping

tool_mapping={tool.name:tool for tool in tools}
tool_mapping

{'multiply': StructuredTool(name='multiply', description='Multiplies two numbers.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x0000029F52F24540>),
 'search': StructuredTool(name='search', description='search the web for a query and return the results', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x0000029F52F242C0>)}

In [13]:
tool_mapping["search"]

StructuredTool(name='search', description='search the web for a query and return the results', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x0000029F52F242C0>)

In [14]:
tool_mapping["search"].invoke({"query":"What is the capital of india?"})

C:\Users\Harshit\AppData\Local\Temp\ipykernel_18004\1333534694.py:9: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily=TavilySearchResults()


'Result for What is the capital of india? is: \n[{\'title\': \'What is the capital of India? States and union territories explained.\', \'url\': \'https://www.usatoday.com/story/news/world/2023/05/24/what-is-the-capital-of-india/70195720007/\', \'content\': \'Want to learn more about the soon-to-be most populous country? Here’s some interesting information about how India is organized.\\n\\n## What is the capital of India?\\n\\nThe capital of India is New Delhi, located in the north-central part of the country to the west of the Yamuna River.\\n\\nCalcutta (now Kolkata, the capital of West Bengal) was the country’s capital until 1911 when King George V declared Delhi the new capital and construction of New Delhi began. [...] When the national government achieved independence in 1947, New Delhi became the capital.\\n\\nMumbai, the state capital of Maharashtra, is often considered the financial capital of India because of its role in the national and international economy.\\n\\n## How ma

In [15]:
tool_mapping[result.tool_calls[0]["name"]].invoke(result.tool_calls[0]["args"])

'Result for current GDP of India is: \n[{\'title\': \'India Becoming An Economic Powerhouse - PIB\', \'url\': \'https://www.pib.gov.in/PressNoteDetails.aspx?NoteId=154660\', \'content\': \'India’s GDP has witnessed a remarkable transformation over the past decade. At current prices, GDP has increased from ₹106.57 lakh crore in 2014–15 to an estimated ₹331.03 lakh crore in 2024–25, an approximate threefold rise in just ten years. In 2024–25 alone, nominal GDP grew by 9.9% over the previous year, while real GDP (at constant prices) increased by 6.5%, reflecting sustained economic momentum. This steep growth reflects the country’s expanding economic base and rising income levels. [...] India, the world’s fourth-largest economy, has emerged as the fastest-growing major economy and is on track to become the world’s third-largest economy with a projected GDP of $7.3 trillion by 2030. India is projected to be world’s fastest growing major economy (6.3% to 6.8% in 2025-26). This transformation

#### Human In Loop Use Case 1: Taking Permission and Proceeding

![alt text](428a6a26-e9b8-4125-b8a9-aab14b692d23.png)

In [16]:
from typing import TypedDict, Sequence, Annotated
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START,END

In [ ]:
# Functions

class AgentState(TypedDict):
    """State for the agent."""
    messages: Annotated[Sequence[BaseMessage],operator.add]

# LLM Calling Function
def invoke_model(state:AgentState):
    messages=state["messages"]
    question=messages[-1]
    response=llm_with_tools.invoke(question)
    return {"messages":[response]}

# Router Function
def router(state:AgentState):
    tool_calls=state["messages"][-1].tool_calls
    if len(tool_calls)>0:
        return "tool" #key name
    else:
        return "end" #key name
    
# Tool Calling Function: Responsible for Asking User for approval to use a tool
def invoke_tool(state:AgentState):
    tool_details=state["messages"][-1].tool_calls
    
    if tool_details is None:
        return Exception("No tool calls found in the last message.")
    
    print(f"Seleted tool: {tool_details[0]['name']}")
    
    if tool_details[0]["name"]=="search":
        response=input(prompt=f"[yes/no] do you want to continue with this expensive web search")
        if response.lower()=="no":
            print("web search discarded by the user. exiting gracefully")
            raise Exception("Web search discarded by the user.")
            
    
    response=tool_mapping[tool_details[0]["name"]].invoke(tool_details[0]["args"])
    return {"messages":[response]}

In [ ]:
# Workflow

graph=StateGraph(AgentState)

graph.add_node("ai_assistant", invoke_model)

# eariler we were using the ToolNode(from the prebuilt library) from list of tool
# but now we have created tool_invoke (custom funtion)
# why we are doing it? -> as a user if we want to take a authority to which i need to give permission for execution

graph.add_node("tool", invoke_tool)

graph.add_conditional_edges("ai_assistant",
                            router,
                            {
                                "tool":"tool", ##with the key tool which value is associated <tool>
                                "end":END
                            }
                            )

graph.add_edge("tool", END)

graph.set_entry_point("ai_assistant")

app=graph.compile()

app

### Human In Loop Use Case 2: INTERRUPT BEFORE (Take User Input and Integrate in Workflow)

![alt text](18bf5d8f-0193-4d25-ae43-8980b58d2220.png)

In [17]:
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

In [18]:
# Functions

llm_with_tools=llm.bind_tools(tools)

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

def ai_assistant(state: AgentState):
    response=llm_with_tools.invoke(state["messages"])
    return {"messages":[response]}

In [19]:
tool_node=ToolNode(tools)
memory=MemorySaver()

# Workflow

graph_builder=StateGraph(AgentState)
graph_builder.add_node("ai_assistant", ai_assistant)
graph_builder.add_node("tools", tool_node)
graph_builder.add_edge(START,"ai_assistant")
graph_builder.add_conditional_edges("ai_assistant",
                                    tools_condition,
                                    )
graph_builder.add_edge("tools", "ai_assistant")

In [20]:
# Introducing the concept of INTERRUPT BEFORE - This stops the flow before the prescribed state


app2=graph_builder.compile(checkpointer=memory,interrupt_before=["tools"])

#### Integrating Asking User For approval before invoking any tool in the flow

In [21]:
config = {"configurable":{"thread_id":"1"}}
response = app2.invoke({"messages":[HumanMessage("What is the current gdp of the china?")]}, config=config)

response

{'messages': [HumanMessage(content='What is the current gdp of the china?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the current GDP of China. I need to figure out the best way to get this information. I remember that I have access to a tool called "search" which can look up information on the web. \n\nFirst, I should use the "search" tool with the query "current GDP of China" to find the most recent data. Once I get the results, I can extract the specific GDP figure and present it to the user. \n\nI should make sure the search query is clear and specific to get accurate results. After obtaining the data, I\'ll format the response to be helpful and informative for the user.\n', 'tool_calls': [{'id': 'rrnm07r1m', 'function': {'arguments': '{"query":"current GDP of China"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 161, 'prompt_tokens'

In [22]:
# Get State - Captures the State

snapshot=app2.get_state(config)
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='What is the current gdp of the china?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the current GDP of China. I need to figure out the best way to get this information. I remember that I have access to a tool called "search" which can look up information on the web. \n\nFirst, I should use the "search" tool with the query "current GDP of China" to find the most recent data. Once I get the results, I can extract the specific GDP figure and present it to the user. \n\nI should make sure the search query is clear and specific to get accurate results. After obtaining the data, I\'ll format the response to be helpful and informative for the user.\n', 'tool_calls': [{'id': 'rrnm07r1m', 'function': {'arguments': '{"query":"current GDP of China"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 1

In [23]:
# Next - Tells what is Next in the state

snapshot.next

('tools',)

In [24]:
# Getting Tool Details

last_message = snapshot.values["messages"][-1]
tool_details = last_message.tool_calls

tool_details

[{'name': 'search',
  'args': {'query': 'current GDP of China'},
  'id': 'rrnm07r1m',
  'type': 'tool_call'}]

In [25]:
# Handling the Human in loop now, asking user what to do and getting approval

if tool_details[0]["name"]== "search":
    user_input = input(prompt=f"[yes/no] do you want to continue with {tool_details[0]['name']}?").lower()
    if user_input=="no":
        print("web tool discarded")
        raise Exception("Web tool discarded by the user.")
    else:
        response=app2.invoke(None,config) # None continues the state(removes interruption)
        print(response)
else:
    response=app2.invoke(None,config)
    print(response)

{'messages': [HumanMessage(content='What is the current gdp of the china?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the current GDP of China. I need to figure out the best way to get this information. I remember that I have access to a tool called "search" which can look up information on the web. \n\nFirst, I should use the "search" tool with the query "current GDP of China" to find the most recent data. Once I get the results, I can extract the specific GDP figure and present it to the user. \n\nI should make sure the search query is clear and specific to get accurate results. After obtaining the data, I\'ll format the response to be helpful and informative for the user.\n', 'tool_calls': [{'id': 'rrnm07r1m', 'function': {'arguments': '{"query":"current GDP of China"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 161, 'prompt_tokens': 

#### Adding Human Input in the flow

In [47]:
config = {"configurable":{"thread_id":"3"}}
response=app2.invoke({"messages":[HumanMessage("What is the current gdp of the japan?")]},config=config)

response

{'messages': [HumanMessage(content='What is the current gdp of the japan?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the current GDP of Japan. I need to figure out how to get that information. I remember that I have access to a tool called "search" which can look up information on the web. So, I should use that tool to find the latest GDP data for Japan.\n\nFirst, I\'ll call the search function with the query "current GDP of Japan". That should bring up the most recent data. Once I get the results, I\'ll extract the GDP figure and present it to the user. I need to make sure the information is up-to-date and accurate, so I\'ll rely on credible sources from the search results.\n\nI should also consider the format of the response. The user probably expects a clear and concise answer, so I\'ll present the GDP number along with the year or the most recent quarter available. If there are multipl

In [48]:
snapshot=app2.get_state(config)
snapshot.next

('tools',)

In [49]:
last_message=snapshot.values["messages"][-1]
last_message.tool_calls

[{'name': 'search',
  'args': {'query': 'current GDP of Japan'},
  'id': '8b19zpde8',
  'type': 'tool_call'}]

In [50]:
tool_call_id=last_message.tool_calls[0]["id"]
tool_call_id

'8b19zpde8'

In [51]:
# Adding User Input into the flow using ToolMessage

new_message=[
    ToolMessage(content="according to the latest data 4.1 trillion USD", tool_call_id=tool_call_id), # This will give the User Input here
]

In [52]:
# Updating the state by apending this new message

app2.update_state(config,
                  {
                      "messages": new_message
                  })

{'configurable': {'thread_id': '3',
  'checkpoint_ns': '',
  'checkpoint_id': '1f071654-532e-6511-8002-cac3fcb6f37b'}}

In [53]:
app2.get_state(config).values["messages"][-1]


ToolMessage(content='according to the latest data 4.1 trillion USD', tool_call_id='8b19zpde8')

In [54]:
app2.invoke(None, config=config)

{'messages': [HumanMessage(content='What is the current gdp of the japan?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the current GDP of Japan. I need to figure out how to get that information. I remember that I have access to a tool called "search" which can look up information on the web. So, I should use that tool to find the latest GDP data for Japan.\n\nFirst, I\'ll call the search function with the query "current GDP of Japan". That should bring up the most recent data. Once I get the results, I\'ll extract the GDP figure and present it to the user. I need to make sure the information is up-to-date and accurate, so I\'ll rely on credible sources from the search results.\n\nI should also consider the format of the response. The user probably expects a clear and concise answer, so I\'ll present the GDP number along with the year or the most recent quarter available. If there are multipl

In [55]:
# Until the above We have integrated the User Message into the flow

app2.invoke({"messages":[HumanMessage("What is the current gdp of the japan?")]},config=config)

{'messages': [HumanMessage(content='What is the current gdp of the japan?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the current GDP of Japan. I need to figure out how to get that information. I remember that I have access to a tool called "search" which can look up information on the web. So, I should use that tool to find the latest GDP data for Japan.\n\nFirst, I\'ll call the search function with the query "current GDP of Japan". That should bring up the most recent data. Once I get the results, I\'ll extract the GDP figure and present it to the user. I need to make sure the information is up-to-date and accurate, so I\'ll rely on credible sources from the search results.\n\nI should also consider the format of the response. The user probably expects a clear and concise answer, so I\'ll present the GDP number along with the year or the most recent quarter available. If there are multipl

In [ ]:
# None makes the flow resume

app2.invoke(None,config=config)

{'messages': [HumanMessage(content='What is the current gdp of the japan?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the current GDP of Japan. I need to figure out how to get that information. I remember that I have access to a tool called "search" which can look up information on the web. So, I should use that tool to find the latest GDP data for Japan.\n\nFirst, I\'ll call the search function with the query "current GDP of Japan". That should bring up the most recent data. Once I get the results, I\'ll extract the GDP figure and present it to the user. I need to make sure the information is up-to-date and accurate, so I\'ll rely on credible sources from the search results.\n\nI should also consider the format of the response. The user probably expects a clear and concise answer, so I\'ll present the GDP number along with the year or the most recent quarter available. If there are multipl